# How does simpy use generators ?

## Goal
- understand Python generator
- understand how Simpy uses generators

## Example from tuto-01

Remember the Process we wrote in the basic Simpy example.

The process uses yield to wait for the end of the visit

In [44]:
from simpy import Environment, Process

class Visitor(Process):  
    def __init__(self, env, name):
        super().__init__(env, self.visit())
        self.name = name

    def visit(self): 
        print(f"{self.name}: Here I am at {self.env.now}") 
        duration = 2
        yield env.timeout(duration)
        print(f"{self.name}: I must leave at {self.env.now}") 

What is this yield keyword ?

yield is the keywork Python uses to create generators.

A generator is an handy way to build an iterator.

Wait, what is an iterator ?

## What is an iterator ?

You actually used iterators before.

For instance, you may go through a list using an iterator. 

In the following case, for will ask an iterator from fruits
and iterate over

In [2]:
fruits = [ "Apple", "Orange", "Banana"]
for word in fruits:
    print(word)

Apple
Orange
Banana


One of the advantages of iterators is that you do not need to know the internals of the collection.

For instance, I may iterate over a set of fruits with the same code.

In [3]:
fruits_set = ( "Apple", "Orange", "Banana" )
for word in fruits_set:
    print(word)

Apple
Orange
Banana


The example above is about words. However, iterators are often used with numbers. 

For instance, generate a list of numbers.

In [4]:
numbers = [ 1, 2, 3, 4 ]
for number in numbers:
    print(number)

sum_numbers = sum(numbers)
print(f"sum {sum_numbers}")

1
2
3
4
sum 10


The modulr Itertools defines a bunch of iterators for common purposes.

Reference: 
https://docs.python.org/3/library/itertools.html

Let us use the iterator repeat to yield an list of the same number repeating 3 times.

In [5]:
from itertools import repeat
for number in repeat(10, 3) :
    print(number)

10
10
10


Some iterators are infinite. 

For instance, the iterator count generates number starting from start and incremented by an amount defined by step.

Say we want to generate odd numbers. As the list is infinite the for expression will loop forever. We will use a different way to get the iterator values. We will read the iterator values one by one using next.

In [6]:
from itertools import count
odd_number = count(1,2)
print(next(odd_number))
print(next(odd_number))
print(next(odd_number))

1
3
5


## How does an iterator work ?

Internally, the for mecanism that goes through the list is similar to the code snippet below. 

The for expression creates an iterator and call next to get each value until there is no more value to read. 
When fruit list has been drained it raises an exception StopIteration. As a result for stops iterating over the list.

In [4]:
import sys
words = [ "Apple", "Orange", "Banana"]
words_iterator = iter(words)
try:
    print(next(words_iterator))
    print(next(words_iterator))
    print(next(words_iterator))
    print(next(words_iterator))
except StopIteration as e:
    # fourth next raises this exception bexause the list has only 3 items
    print("Unexpected error:", sys.exc_info()[0])

Apple
Orange
Banana
Unexpected error: <class 'StopIteration'>


## How to write an iterator ?

Say I want to create a Fruit Basket and iterate over. What does it requires ?

Let us write a very simple class and try to use for to iterrate over.

In [16]:
class FruitBasket:
    def __init__(self):
        self.fruits = [ "Apple", "Orange", "Banana"]

basket = FruitBasket()
try:
    for fruit in basket:
        print(fruit)
except:
    print("Error:", sys.exc_info())

Error: (<class 'TypeError'>, TypeError("'FruitBasket' object is not iterable"), <traceback object at 0x106084880>)


This does not work. It complains about FruitBasket not being iterable.

Hmmm, what does it mean to be iterable ?

Let us refactor this a bit in order to be iterable.

First iterable defines a behavior. It tells that you can loop over the object. There are basically two ways of looping over the object and provide the iterable behavior:
- provide a mean to get the element at a given index and implement getitem
- or provide an iterator and implemen iter

As we define iterable partly in terms or iterator, what is an iterator ?

An iterator allow to loop over the elements without the need of an index. Instead, it provides a way to get the next element and implement a method __next__. As a consequence, an iterator must maintain the state of the loop by itself.

Let us implement FruitBasket so that it is iterable. The function getitem will read the element at a given position in the list (an index).

For will be in charge of computing each index values 
up to the point where index is out of bound.

In [5]:
class FruitBasketWithIterable:
    def __init__(self):
        self.fruits = [ "Apple", "Orange", "Banana"]
        
    def __getitem__(self, n):
        if (n > len(self.fruits)):
            # n is out of the list upper bound
            raise IndexError
        # Otherwise get the nth fruit 
        return self.fruits[n]

basket = FruitBasketWithIterable()
for fruit in basket:
    print(fruit)

Apple
Orange
Banana


By the way, methods named with enclosing __ are special. They provide some well known contract. 

For instance:
- __init__ construct an object of the class type
- __repr__ outputs a representation of the pbject as a string

Let us write the iterable the other way. This one is iterable and rely on the iterator of the internal list.

In [6]:
class FruitBasketWithIterator:
    def __init__(self):
        self.fruits = [ "Apple", "Orange", "Banana"]
        
    def __iter__(self):
        return iter(self.fruits)

basket = FruitBasketWithIterator()
for fruit in basket:
    print(fruit)

Apple
Orange
Banana


Another option is to build a custop iterator. 

What would be a FruitBasket iterator ?

An iterator must provide a __next__ method, keep track of  the state of the loop,  
and is self-iterable (meaning returns self as __iter__ )

In [7]:
class FruitBasketIterator:
    def __init__(self):
        # init items
        self.fruits = [ "Apple", "Orange", "Banana"]
        # init state and boundary
        self.state = 0
        self.limit = len(self.fruits)

    def __iter__(self):
        # self-iterable
        return self

    def __next__(self):
        if self.state >= self.limit:
            # n is out of the list upper bound
            raise StopIteration
        # get the nth value 
        value = self.fruits[self.state]
        # keep track of the state
        self.state += 1
        return value
    
basket = FruitBasketIterator()
for fruit in basket:
    print(fruit)

Apple
Orange
Banana


This way of writing the iterable is required when dealing with infinite lists. 

For instance, let us implement the odd numbers iterator 

In [8]:
class OddNumbers:
    def __init__(self):
        # init 
        self.state = -1

    def __iter__(self):
        return self

    def __next__(self):
        self.state += 2
        return self.state 

odds_iter = OddNumbers()
print(next(odds_iter))
print(next(odds_iter))
print(next(odds_iter))

1
3
5


## And now the generator

Generators provides a convenient way to write iterators.

TODO
The main feature of generator is evaluating the elements on demand. When you call a normal function with a return statement the function is terminated whenever it encounters a return statement. In a function with a yield statement the state of the function is “saved” from the last call and can be picked up the next time you call a generator function.

Let us compare the OddNumbers implementation with this one

In [23]:
def odd_numbers_generator():
    n = 1
    while True:
        yield n
        n += 2
        
odds_gen = odd_numbers_generator()
print(next(odds_gen))
print(next(odds_gen))
print(next(odds_gen))

1
3
5


The generator is written in a more compact way. However it is an iterator. It may be used the same way. 

For instance, iterate over geneated first n numbers, print and sum.

In [10]:
def numbers_first_n(n):
    i = 1
    while i <= n:
        yield i
        i += 1

for number in numbers_first_n(4):
    print(number)

sum_numbers = sum(numbers_first_n(4))
print(f"sum {sum_numbers}")

1
2
3
4
sum 10


You may also iterate over lists

In [48]:
def words_generator(): 
    words = [ "Apple", "Orange", "Banana"]
    for w in words:
        yield w 
        
for w in words_generator():
    print(w)

Apple
Orange
Banana


The generator may be a method in a class.

Please note that in order to be iterable the class must implement __iter__ and __next__. Also do not put the generator into next as it would be recreated each time next is called. The state would be reset and it loops forever. Instead write the generator as a function.

In [47]:
class FruitBasket:
    def __init__(self):
        self.fruits = [ "Apple", "Orange", "Banana"]
        self.iterator = self.generator()

    def __iter__(self):
        return self

    def __next__(self):
        return next(self.iterator)
      
    def generator(self):
        for w in self.fruits:
            yield w 

basket = FruitBasket()
print(next(basket))
for fruit in basket:
    print(fruit)

Apple
Orange
Banana


Please note that the self.generator() returns an iterator not a value.

## How does it relates to Simpy events ?

Here is the process we used in tuto-01 mpdified to be able to trigger the generator. 

In [50]:
from simpy import Environment, Process

class Visitor(Process):  
    def __init__(self, env, name):
        super().__init__(env, self.visit())
        self.name = name

    def visit(self): 
        print(f"{self.name}: Here I am at {self.env.now}") 
        duration = 2
        yield env.timeout(duration)
        print(f"{self.name}: I must leave at {self.env.now}")        
        duration = 2
        yield env.timeout(duration)
        print(f"{self.name}: I must leave at {self.env.now}") 

In [52]:
env = Environment()
# create a visitor
visitor = Visitor(env, "Alice")
iterator = visitor.visit()
try:
    print("1")
    next(iterator)
    print("2")
    next(iterator)
    print("3")
    next(iterator)
    print("4")
    next(iterator)
except:
    print("Error:", sys.exc_info())

1
Alice: Here I am at 0
2
Alice: I must leave at 0
3
Alice: I must leave at 0
Error: (<class 'StopIteration'>, StopIteration(), <traceback object at 0x1125adc00>)


The generator will let the process define what comes next. 
The environment will use the generator as a standard 
interface to get the next element.

In [ ]:
TODO pas la bonne explication

TODO separer 1 yield 2 yield

TODO process is not iterble

## Multiple yields

What happens if the generator has multiple yields

In [27]:
def fruits_generator():
    fruits = [ "Apple", "Orange", "Banana"]
    n = 0
    while n < len(fruits):
        yield n
        yield fruits[n]
        n += 1
        
fruits = fruits_generator()
for fruit in fruits:
    print("-")
    print(fruit)

-
0
-
Apple
-
1
-
Orange
-
2
-
Banana


What happens if the generator chain another generator

In [35]:
def numbers_generator():
    i = 0
    while i < 3:
        yield i
        i += 1
        
def fruits_generator():
    fruits = [ "Apple", "Orange", "Banana"]
    gen = numbers_generator()
    while True:   
        try:
            yield fruits[next(gen) ]
        except:
            n=99
            
for fruit in fruits_generator():
    print(fruit)

Apple
Orange
Banana


KeyboardInterrupt: 